In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
sc = MinMaxScaler(feature_range=(0,1)) #normalisasi dari 0 - 
# importing libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout

In [2]:
data_path = "../data_after_prep"

In [3]:
list_data = os.listdir(data_path)
list_data

['ADRO2001-2407.csv',
 'AMMN2001-2407.csv',
 'AMRT2001-2407.csv',
 'ASII2001-2407.csv',
 'BBCA2001-2407.csv',
 'GOTO2001-2407.csv',
 'KLBF2001-2407.csv',
 'MAPI2001-2407.csv',
 'TLKM2001-2407.csv']

In [4]:
ADRO = pd.read_csv(data_path + "/" + list_data[0], index_col='date', parse_dates=['date'])
AMMN = pd.read_csv(data_path + "/" + list_data[1], index_col='date', parse_dates=['date'])
AMRT = pd.read_csv(data_path + "/" + list_data[2], index_col='date', parse_dates=['date'])
ASII = pd.read_csv(data_path + "/" + list_data[3], index_col='date', parse_dates=['date'])
BBCA = pd.read_csv(data_path + "/" + list_data[4], index_col='date', parse_dates=['date'])
GOTO = pd.read_csv(data_path + "/" + list_data[5], index_col='date', parse_dates=['date'])
KLBF = pd.read_csv(data_path + "/" + list_data[6], index_col='date', parse_dates=['date'])
MAPI = pd.read_csv(data_path + "/" + list_data[7], index_col='date', parse_dates=['date'])
TLKM = pd.read_csv(data_path + "/" + list_data[8], index_col='date', parse_dates=['date'])



In [5]:
#Method
# Some functions to help out with
def plot_predictions(stock_title,test,predicted, title):
    plt.plot(test, color='red',label='Real '+ stock_title+' Stock Price')
    plt.plot(predicted, color='blue',label='Predicted '+ stock_title+' Stock Price')
    plt.title(stock_title + ' Stock Price Prediction '+title)
    plt.xlabel('Time')
    plt.ylabel(stock_title +' Stock Price')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))
    return rmse

def return_rmse_norm(test, pred):
    len_pred = len(pred)
    #pred = pred.reshape(len_pred)
    total = np.concatenate((pred, test))
    #total = total.reshape(-1,1)
    total = sc.fit_transform(total)
    arr_pred = total[len_pred:]
    arr_test = total[:len_pred]
    rmse = math.sqrt(mean_squared_error(arr_test, arr_pred))
    print("The root mean squared error after normalization is {}.".format(rmse))
    return rmse

In [6]:
def short_term_train(data, name, start_date, med_date, end_date, epoch, unit, optimizer, batch_size):
    data_short_train = data[start_date:med_date].iloc[:,3:4].values
    data_short_test = data[med_date:end_date].iloc[:,3:4].values

    #Scaling the training set
    sc = MinMaxScaler(feature_range=(0,1))
    training_set_scaled = sc.fit_transform(data_short_train)
    len_train = len(data_short_train)
    len_test = len(data_short_test)
    print(len_train, len_test)

    # Since LSTMs store short term memory state, we create a data structure with 5 timesteps and 1 output
    # So for each element of training set, we have 5 previous training set elements 
    X_train = []
    y_train = []
    for i in range(5,len_train):
        X_train.append(training_set_scaled[i-5:i,0])
        y_train.append(training_set_scaled[i,0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    # Reshaping X_train for efficient modelling
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

    # The LSTM architecture
    regressorLSTM = Sequential()
    # First LSTM layer with Dropout regularisation
    regressorLSTM.add(LSTM(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1)))
    regressorLSTM.add(Dropout(0.2))
    # Second LSTM layer
    regressorLSTM.add(LSTM(units=unit, return_sequences=True))
    regressorLSTM.add(Dropout(0.2))
    # Third LSTM layer
    regressorLSTM.add(LSTM(units=unit, return_sequences=True))
    regressorLSTM.add(Dropout(0.2))
    # Fourth LSTM layer
    regressorLSTM.add(LSTM(units=unit))
    regressorLSTM.add(Dropout(0.2))
    # The output layer
    regressorLSTM.add(Dense(units=1))

    # Compiling the RNN
    regressorLSTM.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    regressorLSTM.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)
    # Save Model
    regressorLSTM.save("../model_dump/lstm_short_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h5")

    # The GRU architecture
    regressorGRU = Sequential()
    # First GRU layer with Dropout regularisation
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Second GRU layer
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Third GRU layer
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Fourth GRU layer
    regressorGRU.add(GRU(units=unit, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # The output layer
    regressorGRU.add(Dense(units=1))
    # Compiling the RNN
    regressorGRU.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    regressorGRU.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)
    # Save Model
    regressorGRU.save("../model_dump/gru_short_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h5")
    



    # initializing the RNN
    regressorRNN = Sequential()

    # adding first RNN layer and dropout regulatization
    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True, 
                input_shape = (X_train.shape[1],1))
                )

    regressorRNN.add(
        Dropout(0.2)
                )


    # adding second RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding third RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding fourth RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding the output layer
    regressorRNN.add(Dense(units = 1))

    # compiling RNN
    regressorRNN.compile(
        optimizer = optimizer, 
        loss = "mean_squared_error",
        metrics = ["accuracy"])

    # fitting the RNN
    regressorRNN.fit(X_train, y_train, epochs = epoch, batch_size = batch_size)
    # Save Model
    regressorRNN.save("../model_dump/rnn_short_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h5")

    dataset_total = pd.concat((data["close"][start_date:med_date],data["close"][med_date:end_date]),axis=0)
    inputs = dataset_total[len(dataset_total)-len(data_short_test) - 5:].values
    inputs = inputs.reshape(-1,1)
    inputs  = sc.transform(inputs)

    # Preparing X_test and predicting the prices
    X_test = []
    for i in range(5,len_test+5):
        X_test.append(inputs[i-5:i,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    LSTM_predicted_stock_price = regressorLSTM.predict(X_test)
    LSTM_predicted_stock_price = sc.inverse_transform(LSTM_predicted_stock_price)
    GRU_predicted_stock_price = regressorGRU.predict(X_test)
    GRU_predicted_stock_price = sc.inverse_transform(GRU_predicted_stock_price)
    RNN_predicted_stock_price = regressorRNN.predict(X_test)
    RNN_predicted_stock_price = sc.inverse_transform(RNN_predicted_stock_price)
    # Visualize the data
    # Plot the data
    train = data[start_date:med_date]
    valid = data[med_date:end_date]
    valid['pred_lstm'] = LSTM_predicted_stock_price
    valid['pred_gru'] = GRU_predicted_stock_price
    valid['pred_rnn'] = RNN_predicted_stock_price
    # Visualize the data
    plt.figure(figsize=(16,6))
    plt.title('Model LSTM, GRU, and RNN in ' + name +' Short Term Predictions')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price Rupiah (Rp)', fontsize=18)
    plt.plot(train['close'])
    plt.plot(valid['close'])
    plt.plot(valid[['pred_lstm', 'pred_gru', 'pred_rnn']])
    plt.legend(['train', 'val', 'pred_lstm', 'pred_gru', 'pred_rnn'], loc='lower right')
    plt.savefig('../result_paper_tuning/'+name+'_short_'+str(optimizer)+'_'+str(batch_size)+'_'+str(epoch)+'_'+str(unit)+'.png')
    # Evaluating the LSTM
    rmse_data_short_LSTM = return_rmse(data_short_test, LSTM_predicted_stock_price)
    norm_rmse_data_short_LSTM = return_rmse_norm(data_short_test, LSTM_predicted_stock_price)
    # Evaluating the RNN
    rmse_data_short_GRU = return_rmse(data_short_test, GRU_predicted_stock_price)
    norm_rmse_data_short_GRU = return_rmse_norm(data_short_test, GRU_predicted_stock_price)
    # Evaluating the sequence
    rmse_data_short_RNN = return_rmse(data_short_test, RNN_predicted_stock_price)
    norm_rmse_data_short_RNN = return_rmse_norm(data_short_test, RNN_predicted_stock_price)
    return(rmse_data_short_LSTM, rmse_data_short_GRU, rmse_data_short_RNN, norm_rmse_data_short_LSTM, norm_rmse_data_short_GRU, norm_rmse_data_short_RNN)



In [7]:
def med_term_train(data, name, start_date, med_date, end_date, epoch, unit, optimizer, batch_size):
    data_med_train = data[start_date:med_date].iloc[:,3:4].values
    data_med_test = data[med_date:end_date].iloc[:,3:4].values

    #Scaling the training set
    sc = MinMaxScaler(feature_range=(0,1))
    training_set_scaled = sc.fit_transform(data_med_train)
    len_train = len(data_med_train)
    len_test = len(data_med_test)
    print(len_train, len_test)

    # Since LSTMs store med term memory state, we create a data structure with 20 timesteps and 1 output
    # So for each element of training set, we have 20 previous training set elements 
    X_train = []
    y_train = []
    for i in range(20,len_train):
        X_train.append(training_set_scaled[i-20:i,0])
        y_train.append(training_set_scaled[i,0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    # Reshaping X_train for efficient modelling
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

    # The LSTM architecture
    regressorLSTM = Sequential()
    # First LSTM layer with Dropout regularisation
    regressorLSTM.add(LSTM(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1)))
    regressorLSTM.add(Dropout(0.2))
    # Second LSTM layer
    regressorLSTM.add(LSTM(units=unit, return_sequences=True))
    regressorLSTM.add(Dropout(0.2))
    # Third LSTM layer
    regressorLSTM.add(LSTM(units=unit, return_sequences=True))
    regressorLSTM.add(Dropout(0.2))
    # Fourth LSTM layer
    regressorLSTM.add(LSTM(units=unit))
    regressorLSTM.add(Dropout(0.2))
    # The output layer
    regressorLSTM.add(Dense(units=1))

    # Compiling the RNN
    regressorLSTM.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    regressorLSTM.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)
    # Save Model
    regressorLSTM.save("../model_dump/lstm_med_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h20")

    # The GRU architecture
    regressorGRU = Sequential()
    # First GRU layer with Dropout regularisation
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Second GRU layer
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Third GRU layer
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Fourth GRU layer
    regressorGRU.add(GRU(units=unit, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # The output layer
    regressorGRU.add(Dense(units=1))
    # Compiling the RNN
    regressorGRU.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    regressorGRU.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)
    # Save Model
    regressorGRU.save("../model_dump/gru_med_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h20")
    



    # initializing the RNN
    regressorRNN = Sequential()

    # adding first RNN layer and dropout regulatization
    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True, 
                input_shape = (X_train.shape[1],1))
                )

    regressorRNN.add(
        Dropout(0.2)
                )


    # adding second RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding third RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding fourth RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding the output layer
    regressorRNN.add(Dense(units = 1))

    # compiling RNN
    regressorRNN.compile(
        optimizer = optimizer, 
        loss = "mean_squared_error",
        metrics = ["accuracy"])

    # fitting the RNN
    regressorRNN.fit(X_train, y_train, epochs = epoch, batch_size = batch_size)
    # Save Model
    regressorRNN.save("../model_dump/rnn_med_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h20")

    dataset_total = pd.concat((data["close"][start_date:med_date],data["close"][med_date:end_date]),axis=0)
    inputs = dataset_total[len(dataset_total)-len(data_med_test) - 20:].values
    inputs = inputs.reshape(-1,1)
    inputs  = sc.transform(inputs)

    # Preparing X_test and predicting the prices
    X_test = []
    for i in range(20,len_test+20):
        X_test.append(inputs[i-20:i,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    LSTM_predicted_stock_price = regressorLSTM.predict(X_test)
    LSTM_predicted_stock_price = sc.inverse_transform(LSTM_predicted_stock_price)
    GRU_predicted_stock_price = regressorGRU.predict(X_test)
    GRU_predicted_stock_price = sc.inverse_transform(GRU_predicted_stock_price)
    RNN_predicted_stock_price = regressorRNN.predict(X_test)
    RNN_predicted_stock_price = sc.inverse_transform(RNN_predicted_stock_price)
    # Visualize the data
    # Plot the data
    train = data[start_date:med_date]
    valid = data[med_date:end_date]
    valid['pred_lstm'] = LSTM_predicted_stock_price
    valid['pred_gru'] = GRU_predicted_stock_price
    valid['pred_rnn'] = RNN_predicted_stock_price
    # Visualize the data
    plt.figure(figsize=(16,6))
    plt.title('Model LSTM, GRU, and RNN in ' + name +' med Term Predictions')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price Rupiah (Rp)', fontsize=18)
    plt.plot(train['close'])
    plt.plot(valid['close'])
    plt.plot(valid[['pred_lstm', 'pred_gru', 'pred_rnn']])
    plt.legend(['train', 'val', 'pred_lstm', 'pred_gru', 'pred_rnn'], loc='lower right')
    plt.savefig('../result_paper_tuning/'+name+'_med_'+str(optimizer)+'_'+str(batch_size)+'_'+str(epoch)+'_'+str(unit)+'.png')
    # Evaluating the LSTM
    rmse_data_med_LSTM = return_rmse(data_med_test, LSTM_predicted_stock_price)
    norm_rmse_data_med_LSTM = return_rmse_norm(data_med_test, LSTM_predicted_stock_price)
    # Evaluating the RNN
    rmse_data_med_GRU = return_rmse(data_med_test, GRU_predicted_stock_price)
    norm_rmse_data_med_GRU = return_rmse_norm(data_med_test, GRU_predicted_stock_price)
    # Evaluating the sequence
    rmse_data_med_RNN = return_rmse(data_med_test, RNN_predicted_stock_price)
    norm_rmse_data_med_RNN = return_rmse_norm(data_med_test, RNN_predicted_stock_price)
    return(rmse_data_med_LSTM, rmse_data_med_GRU, rmse_data_med_RNN, norm_rmse_data_med_LSTM, norm_rmse_data_med_GRU, norm_rmse_data_med_RNN)



In [8]:
def long_term_train(data, name, year, epoch, unit, optimizer, batch_size):
    data_long_train = data[:year].iloc[:,3:4].values
    data_long_test = data[year:].iloc[:,3:4].values

    #Scaling the training set
    sc = MinMaxScaler(feature_range=(0,1))
    training_set_scaled = sc.fit_transform(data_long_train)
    len_train = len(data_long_train)
    len_test = len(data_long_test)
    print(len_train, len_test)

    # Since LSTMs store long term memory state, we create a data structure with 5 timesteps and 1 output
    # So for each element of training set, we have 5 previous training set elements 
    X_train = []
    y_train = []
    for i in range(60,len_train):
        X_train.append(training_set_scaled[i-60:i,0])
        y_train.append(training_set_scaled[i,0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    # Reshaping X_train for efficient modelling
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

    # The LSTM architecture
    regressorLSTM = Sequential()
    # First LSTM layer with Dropout regularisation
    regressorLSTM.add(LSTM(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1)))
    regressorLSTM.add(Dropout(0.2))
    # Second LSTM layer
    regressorLSTM.add(LSTM(units=unit, return_sequences=True))
    regressorLSTM.add(Dropout(0.2))
    # Third LSTM layer
    regressorLSTM.add(LSTM(units=unit, return_sequences=True))
    regressorLSTM.add(Dropout(0.2))
    # Fourth LSTM layer
    regressorLSTM.add(LSTM(units=unit))
    regressorLSTM.add(Dropout(0.2))
    # The output layer
    regressorLSTM.add(Dense(units=1))

    # Compiling the RNN
    regressorLSTM.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    regressorLSTM.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)
    # Save Model
    regressorLSTM.save("../model_dump/lstm_long_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h5")

    # The GRU architecture
    regressorGRU = Sequential()
    # First GRU layer with Dropout regularisation
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Second GRU layer
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Third GRU layer
    regressorGRU.add(GRU(units=unit, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Fourth GRU layer
    regressorGRU.add(GRU(units=unit, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # The output layer
    regressorGRU.add(Dense(units=1))
    # Compiling the RNN
    regressorGRU.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    regressorGRU.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)
    # Save Model
    regressorGRU.save("../model_dump/gru_long_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h5")

     # initializing the RNN
    regressorRNN = Sequential()

    # adding first RNN layer and dropout regulatization
    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True, 
                input_shape = (X_train.shape[1],1))
                )

    regressorRNN.add(
        Dropout(0.2)
                )


    # adding second RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding third RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit, 
                activation = "tanh", 
                return_sequences = True)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding fourth RNN layer and dropout regulatization

    regressorRNN.add(
        SimpleRNN(units = unit)
                )

    regressorRNN.add(
        Dropout(0.2)
                )

    # adding the output layer
    regressorRNN.add(Dense(units = 1))

    # compiling RNN
    regressorRNN.compile(
        optimizer = optimizer, 
        loss = "mean_squared_error",
        metrics = ["accuracy"])

    # fitting the RNN
    regressorRNN.fit(X_train, y_train, epochs = epoch, batch_size = batch_size)
    # Save Model
    regressorRNN.save("../model_dump/rnn_long_"+name+"_"+str(optimizer)+"_"+str(epoch)+"_"+str(unit)+"_"+str(batch_size)+".h5")



    dataset_total = pd.concat((data["close"][:year],data["close"][year:]),axis=0)
    inputs = dataset_total[len(dataset_total)-len(data_long_test) - 60:].values
    inputs = inputs.reshape(-1,1)
    inputs  = sc.transform(inputs)

    # Preparing X_test and predicting the prices
    X_test = []
    for i in range(60,len_test+60):
        X_test.append(inputs[i-60:i,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    LSTM_predicted_stock_price = regressorLSTM.predict(X_test)
    LSTM_predicted_stock_price = sc.inverse_transform(LSTM_predicted_stock_price)
    GRU_predicted_stock_price = regressorGRU.predict(X_test)
    GRU_predicted_stock_price = sc.inverse_transform(GRU_predicted_stock_price)
    RNN_predicted_stock_price = regressorRNN.predict(X_test)
    RNN_predicted_stock_price = sc.inverse_transform(RNN_predicted_stock_price)

    # Visualize the data
    # Plot the data
    train = data[:year]
    valid = data[year:]
    valid['pred_lstm'] = LSTM_predicted_stock_price
    valid['pred_gru'] = GRU_predicted_stock_price
    valid['pred_rnn'] = RNN_predicted_stock_price
    # Visualize the data
    plt.figure(figsize=(16,6))
    plt.title('Model LSTM, GRU, and RNN in ' + name +' Long Term Predictions')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price Rupiah (Rp)', fontsize=18)
    plt.plot(train['close'])
    plt.plot(valid['close'])
    plt.plot(valid[['pred_lstm', 'pred_gru', 'pred_rnn']])
    plt.legend(['train', 'val', 'pred_lstm', 'pred_gru', 'pred_rnn'], loc='lower right')
    plt.savefig('../result_paper_tuning/'+name+'_long_'+str(optimizer)+'_'+str(batch_size)+'_'+str(epoch)+'_'+str(unit)+'.png')
    # Evaluating the LSTM
    rmse_data_long_LSTM = return_rmse(data_long_test, LSTM_predicted_stock_price)
    norm_rmse_data_long_LSTM = return_rmse_norm(data_long_test, LSTM_predicted_stock_price)
    # Evaluating the RNN
    rmse_data_long_GRU = return_rmse(data_long_test, GRU_predicted_stock_price)
    norm_rmse_data_long_GRU = return_rmse_norm(data_long_test, GRU_predicted_stock_price)
    # Evaluating the sequence
    rmse_data_long_RNN = return_rmse(data_long_test, RNN_predicted_stock_price)
    norm_rmse_data_long_RNN = return_rmse_norm(data_long_test, RNN_predicted_stock_price)
    return(rmse_data_long_LSTM, rmse_data_long_GRU, rmse_data_long_RNN, norm_rmse_data_long_LSTM, norm_rmse_data_long_GRU, norm_rmse_data_long_RNN)



In [9]:
#short term data
start_date = "2024-07-01"
med_date = "2024-07-19"
end_date = "2024-07-31"

#long term data
year = "2023"

## Tuning Hyperparameter
- optimizer = SGD, Adam, RMSProp
- epoch = 50, 100
- batch size =  32, 64, 128
- unit = 50, 100


In [10]:
def train_short(data, name, start_date, med_date, end_date, epoch, unit, batch_size, optimizer):
    rmse_LSTM, rmse_GRU, rmse_RNN, rmse_norm_LSTM, rmse_norm_GRU, rmse_norm_RNN =  short_term_train(data=data, name=name, start_date=start_date, med_date=med_date, end_date=end_date, epoch=epoch, unit=unit, batch_size=batch_size, optimizer= optimizer) 
    dic_res_tuning =  {'stock' : [], 'term' : [], 'algorithm': [], 'rmse' : [], 'rmse_norm' : [], 'epoch' : [], 'unit': [], 'optimizer' : [], 'batch_size' : []}
    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('short')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)

    dic_res_tuning['algorithm'].append('LSTM')
    dic_res_tuning['rmse'].append(rmse_LSTM)
    dic_res_tuning['rmse_norm'].append(rmse_norm_LSTM)

    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('short')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)

    dic_res_tuning['algorithm'].append('GRU')
    dic_res_tuning['rmse'].append(rmse_GRU)
    dic_res_tuning['rmse_norm'].append(rmse_norm_GRU)


    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('short')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)
    
    dic_res_tuning['algorithm'].append('RNN')
    dic_res_tuning['rmse'].append(rmse_RNN)
    dic_res_tuning['rmse_norm'].append(rmse_norm_RNN)
    
    # Make data frame of above data
    df = pd.DataFrame(dic_res_tuning)
    # append data frame to CSV file
    df.to_csv('../result_paper_tuning/result_short_tuning.csv', mode='a', header=False, index=False)
 



In [11]:
def train_medium(data, name, start_date, med_date, end_date, epoch, unit, batch_size, optimizer):
    rmse_LSTM, rmse_GRU, rmse_RNN, rmse_norm_LSTM, rmse_norm_GRU, rmse_norm_RNN =  med_term_train(data=data, name=name, start_date=start_date, med_date=med_date, end_date=end_date, epoch=epoch, unit=unit, batch_size=batch_size, optimizer= optimizer) 
    dic_res_tuning =  {'stock' : [], 'term' : [], 'algorithm': [], 'rmse' : [], 'rmse_norm' : [], 'epoch' : [], 'unit': [], 'optimizer' : [], 'batch_size' : []}
    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('medium')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)

    dic_res_tuning['algorithm'].append('LSTM')
    dic_res_tuning['rmse'].append(rmse_LSTM)
    dic_res_tuning['rmse_norm'].append(rmse_norm_LSTM)

    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('medium')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)

    dic_res_tuning['algorithm'].append('GRU')
    dic_res_tuning['rmse'].append(rmse_GRU)
    dic_res_tuning['rmse_norm'].append(rmse_norm_GRU)


    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('medium')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)
    
    dic_res_tuning['algorithm'].append('RNN')
    dic_res_tuning['rmse'].append(rmse_RNN)
    dic_res_tuning['rmse_norm'].append(rmse_norm_RNN)
    
    # Make data frame of above data
    df = pd.DataFrame(dic_res_tuning)
    # append data frame to CSV file
    df.to_csv('../result_paper_tuning/result_medium_tuning.csv', mode='a', header=False, index=False)
 



In [12]:
def train_long(data, name, year, epoch, unit, batch_size, optimizer):
    rmse_LSTM, rmse_GRU, rmse_RNN, rmse_norm_LSTM, rmse_norm_GRU, rmse_norm_RNN =  long_term_train(data=data, name=name, year=year, epoch=epoch, unit=unit, batch_size=batch_size, optimizer=optimizer) 
    dic_res_tuning =  {'stock' : [], 'term' : [], 'algorithm': [], 'rmse' : [], 'rmse_norm' : [], 'epoch' : [], 'unit': [], 'optimizer' : [], 'batch_size' : []}
    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('long')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)

    dic_res_tuning['algorithm'].append('LSTM')
    dic_res_tuning['rmse'].append(rmse_LSTM)
    dic_res_tuning['rmse_norm'].append(rmse_norm_LSTM)

    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('long')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)

    dic_res_tuning['algorithm'].append('GRU')
    dic_res_tuning['rmse'].append(rmse_GRU)
    dic_res_tuning['rmse_norm'].append(rmse_norm_GRU)


    dic_res_tuning['stock'].append(name)
    dic_res_tuning['term'].append('long')
    dic_res_tuning['epoch'].append(epoch)
    dic_res_tuning['unit'].append(unit)
    dic_res_tuning['optimizer'].append(optimizer)
    dic_res_tuning['batch_size'].append(batch_size)
    
    dic_res_tuning['algorithm'].append('RNN')
    dic_res_tuning['rmse'].append(rmse_RNN)
    dic_res_tuning['rmse_norm'].append(rmse_norm_RNN)
    
    # Make data frame of above data
    df = pd.DataFrame(dic_res_tuning)
    # append data frame to CSV file
    df.to_csv('../result_paper_tuning/result_long_tuning.csv', mode='a')
 



### Short Term Predictions

In [14]:
start_date = "2023-01-01"
med_date = "2023-09-30"
end_date = "2023-12-31"
stock_name = ['ASII', 'AMRT', 'ADRO', 'MAPI', 'KLBF', 'TLKM']
try_epoch = [50, 100]
try_batch = [32, 64]
try_unit = [50, 100]
try_optimizer = ['rmsprop', 'adam', 'sgd']
for n_epoch in try_epoch:
    for n_unit in try_unit:
        for n_batch_size in try_batch:
            for n_optimizer in try_optimizer:
                for n_name in stock_name:
                    if n_name == 'ASII':
                        n_data = ASII
                    elif n_name == 'AMRT':
                        n_data = AMRT
                    elif n_name == 'ADRO':
                        n_data = ADRO
                    elif n_name == 'MAPI':
                        n_data = MAPI
                    elif n_name == 'KLBF':
                        n_data = KLBF
                    elif n_name == 'TLKM':
                        n_data = TLKM
                    train_short(n_data, n_name, start_date, med_date, end_date, n_epoch, n_unit, n_batch_size, n_optimizer)
                

176 63
Epoch 1/50
6/6 [==============================] - 7s 14ms/step - loss: 0.3814
Epoch 2/50
6/6 [==============================] - 0s 13ms/step - loss: 0.1141
Epoch 3/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0413
Epoch 4/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0369
Epoch 5/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0380
Epoch 6/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0294
Epoch 7/50
6/6 [==============================] - 0s 10ms/step - loss: 0.0246
Epoch 8/50
6/6 [==============================] - 0s 11ms/step - loss: 0.0281
Epoch 9/50
6/6 [==============================] - 0s 11ms/step - loss: 0.0219
Epoch 10/50
6/6 [==============================] - 0s 10ms/step - loss: 0.0226
Epoch 11/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0176
Epoch 12/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0215
Epoch 13/50
6/6 [==============================] - 0s 

c:\Users\ThinkPad\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
6/6 [==============================] - 10s 16ms/step - loss: 0.1528
Epoch 2/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0255
Epoch 3/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0242
Epoch 4/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0302
Epoch 5/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0264
Epoch 6/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0259
Epoch 7/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0173
Epoch 8/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0206
Epoch 9/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0183
Epoch 10/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0241
Epoch 11/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0260
Epoch 12/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0240
Epoch 13/50
6/6 [==============================] - 0s 13ms/s

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3192\2377687654.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['pred_lstm'] = LSTM_predicted_stock_price
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3192\2377687654.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['pred_gru'] = GRU_predicted_stock_price
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3192\2377687654.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

The root mean squared error is 163.22273232263802.
The root mean squared error after normalization is 0.19202674390898594.
The root mean squared error is 105.50276580362375.
The root mean squared error after normalization is 0.12412090094543965.
The root mean squared error is 198.9677264128682.
The root mean squared error after normalization is 0.19486844786266347.
176 63
Epoch 1/50
6/6 [==============================] - 12s 13ms/step - loss: 0.2588
Epoch 2/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0723
Epoch 3/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0391
Epoch 4/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0391
Epoch 5/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0339
Epoch 6/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0359
Epoch 7/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0329
Epoch 8/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0331


...

Epoch 1/100
6/6 [==============================] - 18s 26ms/step - loss: 0.1626
Epoch 2/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0349
Epoch 3/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0263
Epoch 4/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0177
Epoch 5/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0137
Epoch 6/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0116
Epoch 7/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0120
Epoch 8/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0107
Epoch 9/100
6/6 [==============================] - 0s 24ms/step - loss: 0.0107
Epoch 10/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0095
Epoch 11/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0086
Epoch 12/100
6/6 [==============================] - 0s 37ms/step - loss: 0.0087
Epoch 13/100
6/6 [==============================

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3192\2377687654.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['pred_lstm'] = LSTM_predicted_stock_price
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3192\2377687654.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['pred_gru'] = GRU_predicted_stock_price
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3192\2377687654.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

The root mean squared error is 82.47505200134378.
The root mean squared error after normalization is 0.1963691714317709.
The root mean squared error is 53.86699412038379.
The root mean squared error after normalization is 0.12825474790567562.
The root mean squared error is 87.24082820700521.
The root mean squared error after normalization is 0.19766387442911856.
176 63
Epoch 1/100
6/6 [==============================] - 22s 39ms/step - loss: 0.2724
Epoch 2/100
6/6 [==============================] - 0s 39ms/step - loss: 0.0703
Epoch 3/100
6/6 [==============================] - 0s 34ms/step - loss: 0.0468
Epoch 4/100
6/6 [==============================] - 0s 51ms/step - loss: 0.0403
Epoch 5/100
6/6 [==============================] - 0s 35ms/step - loss: 0.0251
Epoch 6/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0273
Epoch 7/100
6/6 [==============================] - 0s 42ms/step - loss: 0.0217
Epoch 8/100
6/6 [==============================] - 0s 37ms/step - loss: 0.

c:\Users\ThinkPad\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/100
6/6 [==============================] - 23s 34ms/step - loss: 0.1378
Epoch 2/100
6/6 [==============================] - 0s 35ms/step - loss: 0.0375
Epoch 3/100
6/6 [==============================] - 0s 35ms/step - loss: 0.0314
Epoch 4/100
6/6 [==============================] - 0s 34ms/step - loss: 0.0181
Epoch 5/100
6/6 [==============================] - 0s 32ms/step - loss: 0.0137
Epoch 6/100
6/6 [==============================] - 0s 35ms/step - loss: 0.0137
Epoch 7/100
6/6 [==============================] - 0s 37ms/step - loss: 0.0116
Epoch 8/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0083
Epoch 9/100
6/6 [==============================] - 0s 43ms/step - loss: 0.0082
Epoch 10/100
6/6 [==============================] - 0s 35ms/step - loss: 0.0083
Epoch 11/100
6/6 [==============================] - 0s 34ms/step - loss: 0.0085
Epoch 12/100
6/6 [==============================] - 0s 34ms/step - loss: 0.0085
Epoch 13/100
6/6 [==============================

### Medium Term Predictions


In [ ]:
start_date = "2023-01-01"
med_date = "2023-12-31"
end_date = "2024-07-31"
stock_name = ['ASII', 'AMRT', 'ADRO', 'MAPI', 'KLBF', 'TLKM']
try_epoch = [50, 100]
try_batch = [32, 64]
try_unit = [50, 100]
try_optimizer = ['rmsprop', 'adam', 'sgd']
for n_epoch in try_epoch:
    for n_unit in try_unit:
        for n_batch_size in try_batch:
            for n_optimizer in try_optimizer:
                for n_name in stock_name:
                    if n_name == 'ASII':
                        n_data = ASII
                    elif n_name == 'AMRT':
                        n_data = AMRT
                    elif n_name == 'ADRO':
                        n_data = ADRO
                    elif n_name == 'MAPI':
                        n_data = MAPI
                    elif n_name == 'KLBF':
                        n_data = KLBF
                    elif n_name == 'TLKM':
                        n_data = TLKM
                    train_medium(n_data, n_name, start_date, med_date, end_date, n_epoch, n_unit, n_batch_size, n_optimizer)
                

### Long Term Predcitions

In [15]:
year = 2023
stock_name = ['ASII', 'AMRT', 'ADRO', 'MAPI', 'KLBF', 'TLKM']
try_epoch = [50, 100]
try_batch = [32, 64]
try_unit = [50, 100]
try_optimizer = ['rmsprop', 'adam', 'sgd']
for n_epoch in try_epoch:
    for n_unit in try_unit:
        for n_batch_size in try_batch:
            for n_optimizer in try_optimizer:
                for n_name in stock_name:
                    if n_name == 'ASII':
                        n_data = ASII
                    elif n_name == 'AMRT':
                        n_data = AMRT
                    elif n_name == 'ADRO':
                        n_data = ADRO
                    elif n_name == 'MAPI':
                        n_data = MAPI
                    elif n_name == 'KLBF':
                        n_data = KLBF
                    elif n_name == 'TLKM':
                        n_data = TLKM
                    train_long(n_data, n_name, year, n_epoch, n_unit, n_batch_size, n_optimizer)
                

NameError: name 'ASII' is not defined